# Playground

## Insructions

To use this playground, follow these steps:

1. Run all the cells till the cell titled "Data"
2. The data cell can be use to create data as per need. You can vary the following parameters
    - classes - decides number of output classes
    - features - decides number of input features
    - size - decides the size of the dataset
    
   The dataset is divided into 3 parts automatically:
    1. Train 70%
    2. Test 20%
    3. Validation 10%
    
   The data is in a zipped format which has to be unzipped using the following code:
   ```
   X,Y = zip(*T_train)
    X = np.array(X)
    Y = np.array(Y)
   ```
    
3. The next cell title "Create Model and Train" is used to create the neural network and train it. Use the following code to create the network. As can be seen, features size is already set as input layer size and classes is set as output layer size.
    ```
    n = NeuralNet()
    n.add(Input(features))
    n.add(Hidden(5,TanH()))
    n.add(Hidden(10,TanH()))
    n.add(Hidden(5,TanH()))
    n.add(Output(classes,Sigmoid()))
    n.train(X,Y,batch_size=50,learning_rate=1,epochs=100,val_set=T_val)
    ```
4. Once the model is trained, you can use it to predict new data. Use the following code to do so:
    ```
    X_test,Y_test = zip(*T_test)
    X_test = np.array(X_test)
    Y_test = np.array(Y_test)

    Y_pred = n.predict(X_test)
    comp= np.sum(np.all(np.equal(Y_pred, Y_test), axis=1))/len(Y_test)
    print(comp)
    ```

## Import

In [1]:
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import random
from IPython.display import display
from math import floor
import time

## Classes and Methods

In [2]:
class Activation(object):
    def __init__(self):
        pass
    
class Sigmoid(Activation):
    def __init__(self):
        Activation.__init__(self)
        
    def map(self,x):
        z = np.exp(-x)
        sig = 1 / (1 + z)

        return sig
    
class TanH(Activation):
    def __init__(self):
        Activation.__init__(self)
        
    def map(self,x):
        z1 = np.exp(x)
        z2 = np.exp(-x)
        tanh = (z1 - z2)/ (z1 + z2)

        return tanh
    
class ReLU(Activation):
    def __init__(self):
        Activation.__init__(self)
    
    def map(self,x):
        return [(n if n > 0 else 0) for n in x]

In [3]:
class Layer(object):
    def __init__(self,size,act: Activation):
        self.size = size
        self.activation = act
        self.next = None
        self.prev = None
        self.x = None
        self.weights = None
        self.bias = None
        self.a = None
    
class Input(Layer):
    def __init__(self,size):
        Layer.__init__(self,size,None)
    
class Output(Layer):
    def __init__(self,size,act: Activation):
        Layer.__init__(self,size,act)
    
class Hidden(Layer):
    def __init__(self,size,act: Activation):
        Layer.__init__(self,size,act)
    

In [4]:
class NeuralNet():
    def __init__(self):
        self.layers = []
    
    def add(self,layer: Layer):
        self.layers.append(layer)
        
        if(self.__depth() > 1):
            last = self.layers[-2]
            layer.prev = last
            last.next = layer
            
            last.weights = np.random.rand(layer.size,last.size)
            
        layer.bias = np.random.rand(1,layer.size)
        layer.a = np.zeros(layer.size)
             
    def __depth(self):
        return len(self.layers)
    
    def __parameters(self):
        total = 0
        for i in range(0,self.__depth()-1):
            total = total + self.layers[i].size * self.layers[i+1].size + self.layers[i+1].size
            
        return total
    
    def summary(self):
        print("The neural network is {} layers deep and has {} parameters.".format(self.__depth(), self.__parameters()))
    
    def train(self,X,Y,learning_rate=0.001,batch_size=100,epochs=10,val_set=None):
        self.__validate(X,Y)
        error = 0
        
        for j in range(epochs):
            start = time.process_time()
            dh = display('Epoch {}'.format(j+1),display_id=True)
            
            for i in range(0,len(X),batch_size):
                error,gradient = self.__forward_pass(X[i:i+batch_size],Y[i:i+batch_size])
                self.__backward_pass(learning_rate,gradient)
                dh.update("Epoch {}  Error:{} Processed: {} out of {}".format(j+1,round(error,3),i+batch_size,len(X)))
            
            if(val_set is not None):
                x_val,y_val = zip(*val_set)
                x_val = np.array(x_val)
                y_val = np.array(y_val)
                
                y_pred = self.predict(x_val)
                comp= np.sum(np.all(np.equal(y_pred, y_val), axis=1))/len(y_val)
 
                start = round(time.process_time() - start,2)
                dh.update("Epoch {}  Error:{} Accuracy: {} Execution time: {}s".format(j+1,round(error,3),comp,start))
            else:
                dh.update("Epoch {}  Error:{} Processed: {} out of {}".format(j+1,round(error,3),i+batch_size,len(X)))
            
    def __forward_pass(self,X,Y):
        error = 0
        gradient = np.zeros(len(Y[0]))
        
        for (x,y) in zip(X,Y):
            for n,l in enumerate(self.layers):
                if(n == 0):
                    l.a = x
                elif (n == self.__depth()- 1):
                    l.x = l.prev.a
                    z = np.add(np.dot(l.prev.weights,l.prev.a),l.bias)[0]
                    l.a = l.activation.map(z)
                    error = error + self.__cost(y,l.a)
                    gradient = (gradient + l.a - y) * self.__derivative(l.a,l.activation)
                else:
                    l.x = l.prev.a
                    z = np.add(np.dot(l.prev.weights,l.prev.a),l.bias)[0]
                    l.a = l.activation.map(z)
            
        error = error / len(X)
        return error,gradient
    
    def __backward_pass(self,eta,gradient):
        delta = []
        for n,l in enumerate(self.layers[::-1]):
            if(n == self.__depth()- 1):
                pass
            elif (n == 0):
                delta.append(gradient)
            else:
                delta.append(np.sum(delta[-1] * l.weights.T,axis=1) * self.__derivative(l.a,l.activation))
                
        delta.reverse()
        
        for n,l in enumerate(self.layers):
            if(n == self.__depth()- 1):
                pass
            else:
                d = eta*delta[n]*(np.ones(l.weights.shape) * l.a).T
                l.weights -= d.T
                l.next.bias -= eta*delta[n]
                
    
    def __cost(self,actual,predicted):
        cost = np.sum(np.power(actual - predicted,2))/2
        return cost
    
    def __derivative(self,val,act):
        if(type(act).__name__ == "Sigmoid"):
            return val * (1 - val)
        
        if(type(act).__name__ == "TanH"):
            return (1 - val) * (1 + val)
        
        if(type(act).__name__ == "ReLU"):
            return [ n > 0 for n in val]
        
    
    def __validate(self,X,Y):
        if(self.__depth() < 3):
            raise Exception("Error: Layers not sufficient to train neural network")
        
        if(not(isinstance(self.layers[0],Input))):
            raise Exception("Error: First layer of the network should be an input layer")
            
        if(not(isinstance(self.layers[-1],Output))):
            raise Exception("Error: Last layer of the network should be an output layer")
           
        assert Y.shape[0] == X.shape[0],"Error: X and Y must have equal number of training instances"
        
        shape = np.shape(X) 
        assert shape[0] > 0,"Error: Input data array is empty"
        assert shape[1] == self.layers[0].size,"Error: Input data dimensions must match with input layer dimension"
        
        shape = np.shape(Y) 
        assert shape[0] > 0,"Error: Output data array is empty"
        assert shape[1] == self.layers[-1].size,"Error: Output data dimensions must match with output layer dimension"            
        
    def predict(self,X):
        y_pred = []
        for x in X:
            for n,l in enumerate(self.layers):
                if(n == 0):
                    l.a = x
                elif (n == self.__depth()- 1):
                    l.x = l.prev.a
                    z = np.add(np.dot(l.prev.weights,l.prev.a),l.bias)[0]
                    l.a = l.activation.map(z)
                    y_pred.append(l.activation.map(z))
                else:
                    l.x = l.prev.a
                    z = np.add(np.dot(l.prev.weights,l.prev.a),l.bias)[0]
                    l.a = l.activation.map(z)
        
        y_pred = np.array(y_pred)
        temp = np.zeros_like(y_pred)
        temp[np.arange(len(y_pred)), y_pred.argmax(1)] = 1
        y_pred = temp           
        return np.array(y_pred)

## Data

In [57]:
%%time
classes = 4
size = 10000
features = 4

X, Y = make_blobs(n_samples=size, centers=classes, n_features=features)
temp = np.zeros([len(Y),classes])
temp[np.arange(len(Y)), Y] = 1
Y = temp
T = list(zip(X,Y))
random.shuffle(T)

T_val = T[:floor(0.1*size)]
T_test = T[-floor(0.2*size):]
T_train = T[floor(0.1*size):floor(0.8*size)]

Wall time: 54 ms


## Create Model and Train

In [58]:
%%time
X,Y = zip(*T_train)
X = np.array(X)
Y = np.array(Y)

n = NeuralNet()
n.add(Input(features))
n.add(Hidden(10,TanH()))
n.add(Hidden(10,TanH()))
n.add(Output(classes,Sigmoid()))
n.train(X,Y,batch_size=50,learning_rate=0.05,epochs=10,val_set=T_val)

'Epoch 1  Error:0.668 Accuracy: 0.511 Execution time: 2.69s'

'Epoch 2  Error:0.379 Accuracy: 0.65 Execution time: 2.64s'

'Epoch 3  Error:0.279 Accuracy: 0.751 Execution time: 2.7s'

'Epoch 4  Error:0.224 Accuracy: 0.751 Execution time: 2.67s'

'Epoch 5  Error:0.155 Accuracy: 0.91 Execution time: 2.61s'

...

'Epoch 7  Error:0.051 Accuracy: 0.975 Execution time: 2.59s'

'Epoch 8  Error:0.04 Accuracy: 0.98 Execution time: 2.69s'

'Epoch 9  Error:0.032 Accuracy: 0.982 Execution time: 2.67s'

'Epoch 10  Error:0.027 Accuracy: 0.986 Execution time: 2.81s'

Wall time: 25.9 s


## Predict

In [59]:
%%time
X_test,Y_test = zip(*T_test)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

Y_pred = n.predict(X_test)
acc= np.sum(np.all(np.equal(Y_pred, Y_test), axis=1))/len(Y_test)
print("Accuracy {}%".format(acc*100))
print("\nConfusion Matrix\n")
print(confusion_matrix(Y_test.argmax(1), Y_pred.argmax(1)))

Accuracy 99.1%

Confusion Matrix

[[497   0   0   0]
 [  0 481   0   0]
 [  0  11 491   0]
 [  0   0   7 513]]
Wall time: 354 ms


## Network Summary

In [60]:
n.summary()

The neural network is 4 layers deep and has 204 parameters.
